# 2 4 - Jayden

In [17]:
import pandas as pd
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
import utils as ut



#Seed for randomness in cross-validation

train2, target2, test2 = ut.read_classification_dataset(2)

k = 10
seed = 10
kf = KFold(n_splits=k, shuffle=True, random_state=seed)

#This was the best performing SVM after messing around with different parameters
svr = SVC(kernel='linear',gamma=0.9)

def pcareduce(data:pd.DataFrame):

    pca = PCA(n_components=10)
    pca.fit(train2)
    return pd.DataFrame(pca.transform(train2))
train2_reduced = pcareduce(train2)
test2_reduced = pcareduce(test2)


sk.metrics.get_scorer_names()
scores = cross_val_score(svr, train2_reduced, target2.values.ravel(), cv=kf, scoring='accuracy')


mean_score = scores.mean()
std_score = scores.std()
print(f"Mean Score: {mean_score:.2f}")
print(f"Standard Deviation of Scores: {std_score:.2f}")

Mean Score: 0.88
Standard Deviation of Scores: 0.06


**Start of Data 4**

KNN

In [7]:
train4, target4, test4 = ut.read_classification_dataset(4)

train4.isna().sum().sum()


# svc4 = SVC(kernel='linear',gamma=0.9)
knnr = KNeighborsClassifier(n_neighbors=5,weights='distance')
k = 10
kf = KFold(n_splits=k,shuffle=True,random_state=42)



scores = cross_val_score(knnr,train4,target4.values.ravel(),cv=kf,scoring='accuracy')

print(f"Mean Score: {scores.mean():.2f}")
print(f"Standard Deviation of Scores: {scores.std():.2f}")


Mean Score: 0.74
Standard Deviation of Scores: 0.04


Random Forest

In [65]:
forest = RandomForestClassifier()
#This takes a long time, all the ones commented out were not important
rf_params = {
"n_estimators":[5,10,20,50,100],
# "n_estimators":[10,50,100,200,500],
"max_depth":[1,2,3,4,5],
# "min_samples_split":[2,5,10],
# "min_samples_leaf":[1,2,4],
# "max_features":['auto','sqrt','log2',None],
# "bootstrap":[True,False]

}

gsvc = GridSearchCV(estimator=forest,param_grid=rf_params,cv=5,scoring='f1_weighted')
gsvc.fit(train4,target4.values.ravel())

forest_optimized = gsvc.best_estimator_

print(forest_optimized)
print("n_estimators",forest_optimized.n_estimators)

scores = cross_val_score(forest_optimized,train4,target4.values.ravel(),cv=10,scoring='f1_weighted')

print(f"F1 Score: {scores.mean():.2f}")
print(f"Standard Deviation of Scores: {scores.std():.2f}")

RandomForestClassifier(max_depth=5)
n_estimators 100
F1 Score: 0.73
Standard Deviation of Scores: 0.12


In [80]:
forest = RandomForestClassifier(n_estimators=50,max_depth=5)

scores = cross_val_score(forest,train4,target4.values.ravel(),cv=10,scoring='f1_weighted')

print(f"Mean Score: {scores.mean():.2f}")
print(f"Standard Deviation of Scores: {scores.std():.2f}")

Mean Score: 0.71
Standard Deviation of Scores: 0.14


LASSO Regression

In [90]:
lasso = Lasso()

params = {
'alpha': [*range(1,200)]
}

# score = "neg_mean_squared_error"
score = "r2"
gscv = GridSearchCV(estimator=lasso,param_grid=params,scoring=score,cv=10)

gscv.fit(train4,target4.values.ravel())
lasso2=gscv.best_estimator_
print(lasso2)
# 'neg_mean_squared_error'
scores = cross_val_score(lasso2,train2_reduced,target2.values.ravel(),cv=kf,scoring=score)

print(f"Mean R Squared Score: {scores.mean():.2f}")
print(f"Standard Deviation of Scores: {scores.std():.2f}")

Lasso(alpha=199)
Mean R Squared Score: -0.14
Standard Deviation of Scores: 0.23
